In [1]:
from math import pi,exp 
from ngsolve import *
from xfem import *
from netgen.occ import *
from ngsolve.webgui import *

importing ngsxfem-2.1.2504


In [2]:
def get_triangle_face(vertices):
    e1 = Segment(vertices[0],vertices[1])
    e2 = Segment(vertices[1],vertices[2])
    e3 = Segment(vertices[2],vertices[0])
    return Face(Wire([e1,e2,e3]))

In [3]:
# Define the bulk region
vertices1 = [(0,1,0),(1,0,0),(0,0,0)]
# vertices2 = [(0,1,0),(1,0,0),(1,1,0)]
Omega1 = get_triangle_face(vertices1)
# Omega2 = get_triangle_face(vertices2)
# Omega1.faces.name = "Omega1"
# Omega2.faces.name = "Omega2"

In [4]:
# Define the bulk and the fracture region
background = Rectangle(1, 1).Face() 
background.edges.name="outer" # 给这个矩形的所有边命名为outer
vertices2 = [(0,1,0),(1,0,0),(1,1,0)]
Omega2 = get_triangle_face(vertices2)

Omega1 = background - Omega2 #做布尔差运算，把内矩形从外矩形中“挖掉”，形成一个有孔的区域。
Omega1.faces.name = "Omega1"
Omega2.faces.name = "Omega2"


In [5]:
# Define names of the edges
Omega1.edges.Min(X).name = "left"
Omega1.edges.Min(Y).name = "bottom"
# Omega1.edges[0].name = 'fracture'
Omega2.edges.Max(X).name = "right"
Omega2.edges.Max(Y).name = "top"
Omega2.edges[0].name = 'fracture'

Omega2.faces.col = (1, 0, 0)
# fracture = Segment(vertices2[0],vertices2[1])

geo = Glue([Omega1, Omega2])

In [6]:
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.5))
Draw(mesh);
print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

('left', 'fracture', 'bottom', 'right', 'top')


In [7]:
help(mesh)

Help on Mesh in module ngsolve.comp object:

class Mesh(pybind11_builtins.pybind11_object)
 |  NGSolve interface to the Netgen mesh. Provides access and functionality
 |  to use the mesh for finite element calculations.
 |
 |  Parameters:
 |
 |  mesh (netgen.Mesh): a mesh generated from Netgen
 |
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  BBBoundaries(...)
 |      BBBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |
 |      Return co dim 3 boundary mesh-region matching the given regex pattern
 |
 |  BBoundaries(...)
 |      BBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |
 |      Return co dim 2 boundary mesh-region matching the given regex pattern
 |
 |  Boundaries(...)
 |      Boundaries(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. Boundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |
 |      Ret

In [23]:
region2 = mesh.Materials("Omega2")
mesh2 = region2.mesh# 用 region2 对整体网格进行切片，得到子网格
print(mesh2.GetBoundaries())
region1 = mesh.Materials("Omega1")
mesh1 = region1.mesh
print(mesh1.GetBoundaries())
for e in mesh1.Elements(BND):
    print(e.vertices)

('left', 'fracture', 'bottom', 'right', 'top')
('left', 'fracture', 'bottom', 'right', 'top')
(V0, V4)
(V4, V1)
(V0, V5)
(V5, V6)
(V6, V2)
(V1, V7)
(V7, V2)
(V2, V8)
(V8, V3)
(V3, V9)
(V9, V0)


In [9]:
# Define the levelset function for the interface
levelset = x+y-1
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,'lsetp1')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [48]:
# Construct the unfitted fem space 
Vh = L2(mesh,order=0,dgjumps=True)
dofs_omega1 = Vh.GetDofs(mesh.Materials("Omega1"))
dofs_omega2 = Vh.GetDofs(mesh.Materials("Omega2"))
Vh1 = Restrict(Vh,dofs_omega1)
Vh2 = Restrict(Vh,dofs_omega2)
ci = CutInfo(mesh,lsetp1)
Vhfbase = H1(mesh,order=1,dirichlet='top|bottom|left|right')
Vhf = Compress(Vhfbase,GetDofsOfElements(Vhfbase,ci.GetElementsOfType(IF)))
Xh = FESpace([Vh1,Vh2,Vhf])

In [49]:
print(Vh1.ndof)
print(Vh1.FreeDofs())
print(Vh2.ndof)
print(Vh2.FreeDofs())
print(Vhf.ndof)
print(Vhf.FreeDofs())

5
0: 11111
5
0: 11111
6
0: 000110


In [115]:
u,v = Xh.TnT()

jump_u1 = u[0] - u[0].Other() 
jump_v1 = v[0] - v[0].Other() 
jump_u2 = u[1] - u[1].Other() 
jump_v2 = v[1] - v[1].Other() 
n = specialcf.normal(2)
mean_du1dn = 0.5*n*(grad(u[0])+grad(u[0].Other()))
mean_dv1dn = 0.5*n*(grad(v[0])+grad(v[0].Other()))
mean_du2dn = 0.5*n*(grad(u[1])+grad(u[1].Other()))
mean_dv2dn = 0.5*n*(grad(v[1])+grad(v[1].Other()))

# jump_u_gamma = u[0]-u[1]
# jump_v_gamma = v[0]-v[1]
#  v[0] = jump_v1
#  v[1] = -jump_v2
jump_u_gamma = jump_u1 + jump_v1
jump_v_gamma = jump_u2 + jump_v2
mean_u_gamma = (u[0]+u[1])/2
mean_v_gamma = (v[0]+v[1])/2

In [123]:
sigma = 10**3
lf = 0.01 # the aperture of the fracture
kf = 1
kfn = 1
eta_gamma = lf/kfn
# beta_gamma = 1/eta_gamma
beta_gamma = 1
ksi = 1
# alpha_gamma = 4/eta_gamma/(2*ksi-1)
alpha_gamma = 1
f = [-2*exp(x+y),-exp(x+y),exp(1)/sqrt(2)]
u_exact = [exp(x+y),exp(x+y)/2 + exp(1)*(1/2 + 3*eta_gamma/sqrt(2)),exp(1)*(1+sqrt(2)*eta_gamma)]
gD = u_exact

In [142]:
gamma_facets = GetFacetsWithNeighborTypes(mesh, a=ci.GetElementsOfType(HASNEG), b=ci.GetElementsOfType(POS),bnd_val_a=False,bnd_val_b=False,use_and=True)
dgamma = dx(skeleton=True, definedonelements=gamma_facets)

Ah = BilinearForm(Xh)

# # A^{DG} term
# Ah += grad(u[0])*grad(v[0])*dx
# Ah += -(mean_dv1dn * jump_u1 + mean_du1dn * jump_v1)*dx(skeleton=True, definedon=mesh.Materials('Omega1'))
# Ah += sigma * jump_u1*jump_v1*dx(skeleton=True,definedon=mesh.Materials('Omega1')) 
# Ah += -(grad(v[0]).Trace()*n*u[0] - grad(u[0]).Trace()*n*v[0])*ds(skeleton=True,definedon=mesh.Boundaries('left|bottom'))
# Ah += sigma*u[0]*v[0]*ds(skeleton=True,definedon=mesh.Boundaries('left|bottom'))

# Ah += grad(u[1])*grad(v[1])*dx
# Ah += -(mean_dv2dn * jump_u2 + mean_du2dn * jump_v2)*dx(skeleton=True, definedon=mesh.Materials('Omega2'))
# Ah += sigma * jump_u2*jump_v2*dx(skeleton=True,definedon=mesh.Materials('Omega2')) 
# Ah += -(grad(v[1]).Trace()*n*u[1] - grad(u[1]).Trace()*n*v[1])*ds(skeleton=True,definedon=mesh.Boundaries('right|top'))
# Ah += sigma*u[1]*v[1]*ds(skeleton=True,definedon=mesh.Boundaries('right|top'))

# # A_gamma term
# Ah += grad(u[2]).Trace()*grad(v[2]).Trace()*ds(definedon=mesh.Boundaries("fracture"))


# I^{DG} term
# Ah += beta_gamma*jump_u_gamma*jump_v_gamma *ds(skeleton=True, definedon=mesh.Boundaries("fracture"))
# Ah += beta_gamma*jump_u_gamma*jump_v_gamma *dgamma
# Ah += alpha_gamma * (mean_u_gamma-u[2])*(mean_v_gamma-v[2]) *ds(skeleton=True, definedon=mesh.Boundaries("fracture"))
Ah += u[2]*v[2]*ds(definedon=mesh.Boundaries("fracture"))
# Ah.Assemble()

In [143]:
Ah.Assemble()

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


In [144]:
print(Ah.mat)

Row 0:   0: 0   2: 0   5: 0   10: 0   12: 0   13: 0   15: 0
Row 1:   1: 0   2: 0   12: 0   13: 0   15: 0
Row 2:   0: 0   1: 0   2: 0   3: 0   10: 0   12: 0   13: 0   14: 0   15: 0
Row 3:   2: 0   3: 0   4: 0   9: 0   11: 0   12: 0   13: 0   14: 0   15: 0
Row 4:   3: 0   4: 0   6: 0   11: 0   13: 0   14: 0   15: 0
Row 5:   0: 0   5: 0   8: 0   10: 0   12: 0   13: 0
Row 6:   4: 0   6: 0   9: 0   11: 0   13: 0   14: 0   15: 0
Row 7:   7: 0   8: 0   13: 0
Row 8:   5: 0   7: 0   8: 0   9: 0   10: 0   13: 0   14: 0
Row 9:   3: 0   6: 0   8: 0   9: 0   11: 0   13: 0   14: 0   15: 0
Row 10:   0: 0   2: 0   5: 0   8: 0   10: 0.157135   12: 0   13: 0.0785674   15: 0
Row 11:   3: 0   4: 0   6: 0   9: 0   11: 0.157135   13: 0   14: 0.0785674   15: 0
Row 12:   0: 0   1: 0   2: 0   3: 0   5: 0   10: 0   12: 0   13: 0   14: 0   15: 0
Row 13:   0: 0   1: 0   2: 0   3: 0   4: 0   5: 0   6: 0   7: 0   8: 0   9: 0   10: 0.0785674   11: 0   12: 0   13: 0.31427   14: 0.0785674   15: 0
Row 14:   2: 0   3: 0

In [135]:
Fh = LinearForm(Xh)

# Fh += f[0] * v[0] * dx + f[1] * v[1] * dx + f[2] * v[2] * ds(definedon=mesh.Boundaries("fracture"))
# # Dirichlet boundary term
# Fh += sigma * v[0] * gD[0] * ds(skeleton=True,definedon=mesh.Boundaries('left|bottom')) \
#       + sigma * v[1] * gD[1] * ds(skeleton=True,definedon=mesh.Boundaries('right|top'))
# Fh += -grad(v[0])*n*gD[0]*ds(skeleton=True,definedon=mesh.Boundaries('left|bottom')) \
#       -grad(v[1])*n*gD[1]*ds(skeleton=True,definedon=mesh.Boundaries('right|top')) \


# gamma_facets = GetFacetsWithNeighborTypes(mesh, a=ci.GetElementsOfType(HASNEG), b=ci.GetElementsOfType(POS),bnd_val_a=False,bnd_val_b=False,use_and=True)
# ba_surround_facets = GetElementsWithNeighborFacets(mesh,gamma_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")  
# Fh += v[1]*ds(definedon=mesh.Boundaries("fracture"))
# Fh += v[0]*dx(skeleton=True, definedonelements=gamma_facets) # fracture
Fh += jump_v1*dx(skeleton=True, definedonelements=gamma_facets) # 
Fh += -jump_v2*dx(skeleton=True, definedonelements=gamma_facets) # fracture
# Fh += v[0] * ds(skeleton=True, definedon=mesh.Materials("Omega1"))
# Fh += v[1] * dx(skeleton=True, definedon=mesh.Materials("Omega2"))
# Fh += v[1] * dx(skeleton=True,definedonelements=ba_surround_facets)

# Fh += v[1] * ds(skeleton=True) # right + top
# Fh += jump_v2*dx(skeleton=True)

# Fh += v[0]*ds(skeleton=True,definedon=mesh.Boundaries("fracture")) # fracture
# Fh += -jump_v2*ds(skeleton=True,definedon=mesh.Boundaries("fracture")) 
Fh.Assemble()


In [136]:
print(Fh.components[0].vec)
print(Fh.components[1].vec)
# print(Fh.components[2].vec)

 0.471405
       0
       0
 0.471405
 0.471405


 0.471405
 0.471405
       0
       0
 0.471405




In [102]:
print(sqrt(5)/6)
print(sqrt(2)/2)
print(sqrt(17)/6)
print(sqrt(2)/3)

0.37267799624996495
0.7071067811865476
0.6871842709362768
0.47140452079103173


In [107]:
uh = GridFunction(Xh)
uh.components[2].vec[0] = u_exact[2]
uh.components[2].vec[1] = u_exact[2]
# uh.components[2].Set(u_exact[2],BND)
print(uh.components[2].vec)


 2.75672
 2.75672
       0
       0
       0
       0




In [108]:
fh = Fh.vec.CreateVector()
fh.data = Fh.vec - Ah.mat * uh.vec
uh.vec.data += Ah.mat.Inverse(Xh.FreeDofs())*fh

In [109]:
Draw(uh.components[0]+uh.components[1],mesh)
# Draw(uh.components[0],mesh)
# Draw(uh.components[1],mesh)
# DrawDC(lsetp1,uh.components[0],uh.components[1],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [110]:
u = GridFunction(Xh)
u.components[0].Set(u_exact[0])
u.components[1].Set(u_exact[1])
Draw(u.components[0]+u.components[1],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [111]:
err = sqrt(Integrate((uh.components[2] - u_exact[2])**2*ds(definedon=mesh.Boundaries("fracture")), mesh=mesh))
print(err)
Draw(uh.components[2],mesh)
uh2 = GridFunction(Vhf)
uh2.Set(u_exact[2])
uh2.vec[2] = 0
uh2.vec[5] = 0
# print(uh2.vec)
Draw(uh2,mesh)

5135614997979.477


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [32]:
print(Vhf.ndof)
print(Vhf.FreeDofs())

12
0: 000011111001


In [44]:
print(Xh.FreeDofs())

0: 11111111111111111111111111111111111111111111111111
50: 1111111111111111111111111111111111000011111001


In [17]:
for el in mesh.Elements():
    print(el.vertices)

(V0, V4, V5)
(V1, V7, V4)
(V4, V7, V5)
(V5, V7, V6)
(V2, V6, V7)
(V0, V5, V9)
(V2, V8, V6)
(V3, V9, V8)
(V5, V8, V9)
(V5, V6, V8)
